In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
#working directory
# working_dir = "/Users/augustus/Desktop/WSWC/WaDE/MappingStatesDataToWaDE2.0/Wyoming/AggregatedAmounts/RawInputData/"
# os.chdir(working_dir)

working_dir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Wyoming/AggregatedAmounts"
os.chdir(working_dir)

In [3]:
# create output dataframe
target_columns = ["ReportingUnitUUID", "ReportingUnitNativeID", "ReportingUnitName",
                  "ReportingUnitTypeCV", "ReportingUnitUpdateDate", "ReportingUnitProductVersion",
                  "StateCV", "EPSGCodeCV", "Geometry"]

outdf = pd.DataFrame(columns=target_columns)

In [4]:
# input file
file = "RawInputData/Agricultural, Municipal, Domestic, and Industrial Use Indecies_Draft_For Delivery to WWDC_2019.0703.xlsx"

In [8]:
# import file to dataframe
df = pd.read_excel(file, header=2)
df = df.rename(columns={'Unnamed: 1':'UseType', 'Unnamed: 2':'Source'})

# Fill nan with 'blank'
df= df.fillna('blank')
df.to_csv("stufffileinput", index=False, encoding = "utf-8")
df

,Unnamed: 0,UseType,Source,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,Water Use by Basin,blank,blank,2000,2001,2002.000000,2003.000000,2004.000000,2005.000000,2006.000000,...,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
1,Bear River Planning Basin,Agricultural Consumptive Use,Total,66151.8,74287.4,60814.170300,80190.234150,91557.115650,102132.197700,90562.174950,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!
2,blank,blank,Surface Water,64817.5,72789,59587.557520,78572.809050,89710.422230,100072.206500,88735.549340,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!
3,blank,blank,Ground Water,1334.27,1498.37,1226.612777,1617.425105,1846.693415,2059.991249,1826.625609,...,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!,#DIV/0!
4,blank,Municipal Use,Total,1040,1026.91,1035.672578,1030.453148,1030.588319,1032.389191,1044.173587,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,blank,blank,Surface Water,2904.42,2857.43,5348.409647,5354.319700,5408.763686,5523.645662,5623.598329,...,0,0,0,0,0,0,0,0,0,0
94,blank,blank,Ground Water,20088.9,20200.5,27403.427500,27566.638960,27810.503620,28138.380940,28527.169030,...,0,0,0,0,0,0,0,0,0,0
95,blank,Industrial Use,Total,233419,254979,258009.735600,247518.935300,251041.408100,253285.547000,271040.797900,...,0,0,0,0,0,0,0,0,0,0
96,blank,blank,Surface Water,171318,170440,164330.058500,164133.018700,169060.735800,159416.413500,160290.419200,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
distribute basin names over blanks
x = df.loc[0, 'Water Use by Basin']

for i, row in df.iterrows():
    y = row['Water Use by Basin']
    if y == 'blank':
        df.at[i, 'Water Use by Basin'] = x
    else:
        x = y

In [ ]:
# only need county names in reporting units
# remove redundancies
df = df.drop_duplicates(subset='Water Use by Basin')
df = df.reset_index(drop=True)

#Todo: ask Adel about deleting statewide row 
indexNames = df[ (df['Water Use by Basin'] == 'blank') | (df['Water Use by Basin'] == 'Statewide Water Use Index')].index
df.drop(indexNames , inplace=True)

outdf["ReportingUnitName"] = df["Water Use by Basin"]

In [ ]:
# Reporting unit native ID:  Auto generate
outdf['ReportingUnitNativeID'] = range(1, len(outdf.index) + 1)

# ReportingUnitUUID:  WY_NativeID
outdf['ReportingUnitUUID'] = outdf.apply(lambda row: "_".join(["WY", str(row['ReportingUnitNativeID'])]), axis=1)

In [ ]:
# hardcoded
outdf.StateCV = "WY"
outdf.ReportingUnitTypeCV = "Basin"

#ToDo:  get hardcoded EPSG code from Adel
#outdf.EPSGCodeCV = "EPSG:4326"

# replace NaN with blank cells
outdf = outdf.replace(np.nan, '')
outdf.head(5)

In [ ]:
print("Exporting csv...")


#write out
out_repunit = 'reportingunits.csv'
outdf.to_csv(out_repunit, index=False, encoding = "utf-8")


print("DONE")